In [20]:
import json
import os
from moo_algorithm.metric import cal_hv
import numpy as np

def cal_HV_algorithm(data_name, number_customers, algorithm):
    data_path = os.path.join(
        "result",
        algorithm,
        f"N{number_customers}",
        f"{data_name}.json"
    )


    with open(data_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    objectives_list = data["history"]["99"]
    hv_value = cal_hv(np.array(objectives_list), ref_point=np.array([100, 500000]))
    return hv_value


In [26]:
algorithms = ['NSGA_II', "NSGA_III" ,"MOEAD", "PFG_MOEA", "AGEA"]

for algor in algorithms:
    hv = cal_HV_algorithm("S042_N200_RC_R50", 200, algor)
    print(f"{algor}: HV = {hv}")    

NSGA_II: HV = 28096343.005115345
NSGA_III: HV = 29791248.72624704
MOEAD: HV = 22532071.72706493
PFG_MOEA: HV = 27775760.84514374
AGEA: HV = 28154751.765239436
